In [ ]:
# Tutorial: https://github.com/terrier-org/cikm2021tutorial/tree/main
import os
import pandas as pd
import pyterrier as pt
import matplotlib.pyplot as plt

In [ ]:
if not pt.started():
    pt.init()

In [ ]:
# Importamos un dataset (corpus) desde el repo de pyTerrier: https://pyterrier.readthedocs.io/en/latest/datasets.html
#
dataset_name = "vaswani"
dataset = pt.datasets.get_dataset(dataset_name)

### Parte 1 - Explorar el corpus

In [ ]:
# Obtenemos el archivo de documentos
dataset.get_corpus()

# Pasamos los documentos a un dataframe
documents = dataset.get_corpus_iter()
df = pd.DataFrame(documents)

In [ ]:
df.shape

In [ ]:
df.head()

### Parte 2 - Indexar el corpus con pyTerrier

In [ ]:
#
index_path = "./vaswani_index"

if os.path.exists(index_path):
    index = pt.IndexFactory.of(index_path+"/data.properties")
else:
    indexer  = pt.TRECCollectionIndexer(index_path, blocks=True)
    indexref = indexer.index(dataset.get_corpus())
    index = pt.IndexFactory.of(indexref)


In [ ]:
# Obtenemos estadísticas del índice
print(index.getCollectionStatistics().toString())

**Exploramos el diccionario (lexicon)**  

In [ ]:
# Obtenemos el diccionario
lex = index.getLexicon()

# Salida:
# término -> id, Nt, TF, maxTF, @
#
# donde:
#       Nt es el DF, número de docs donde aparece el término (sirve para calcular el IDF).
#       TF es la suma de los Tfs de los Nt docs.
#       maxTF es el número total de ocurrencias del término.
#       Los números entre @{} son punteros para Terrier.

for i, kv in enumerate(lex):
    print("%s -> %s" % (kv.getKey(), kv.getValue().toString()))
    #print(kv.getFrequency())
    if (i > 10): break

In [ ]:
# Obtener el término a partir de un id
termid = 3257
#
lee  = lex.getLexiconEntry(termid)
term = lee.getKey()
print (termid, term)

**Exploramos posting lists y documentos en el índice**

In [ ]:
# Obtner la posting list del término ´x´
pointer = index.getLexicon()[term]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen = %d" % posting.getDocumentLength())

In [ ]:
# Términos en un doc
di  = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
#
docid = 9127
#
for posting in di.getPostings(doi.getDocumentEntry(docid)):
    termid = posting.getId()
    lee    = lex.getLexiconEntry(termid)
    print("Término '%s', TF = %d" % (lee.getKey(), posting.getFrequency()))

### Parte 3 - Ejecutar un experimento de recuperación (y evaluación)

In [ ]:
# Obtenemos los 'topics'
topics = dataset.get_topics()
topics

In [ ]:
# Obtenemos los 'qrels'
qrels = dataset.get_qrels()
qrels

**Definimos modelos de recuperación a usar**

In [ ]:
# Búsquedas (más modelos en: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html)
model_tf    = pt.BatchRetrieve(index, wmodel="Tf")
model_tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
model_bm25  = pt.BatchRetrieve(index, wmodel="BM25")
model_dlm   = pt.BatchRetrieve(index, wmodel="DirichletLM") 

**Ejecutamos una búsqueda**

In [ ]:
#%%timeit
# Primero con con bm25 como modelo de RI
model_bm25.search("chemical document")

In [ ]:
# Ahora con con con dlm como modelo de RI
model_dlm.search("chemical document")

In [ ]:
# Varios queries (a partir de un DF)
many_queries = pd.DataFrame([["q1", "chemical document"], ["q2", "first document"]], columns=["qid", "query"])
model_bm25.transform(many_queries)
model_bm25(many_queries)

In [ ]:
many_queries

### Parte 4 - Experimentos completos de Recuperación y Evaluación

In [ ]:
# Un "Experiment" se define a partir de los modelos que queremos evaluar y los datos a usar. 
# Más opciones: https://pyterrier.readthedocs.io/en/latest/experiments.html

pt.Experiment(
    [model_bm25, model_dlm],                           # Que estamos evaluando? (modelos)
    dataset.get_topics(),                              # Qué queries usamos?
    dataset.get_qrels(),                               # Qué juicios de relevancia?
    eval_metrics=[ "P_5", "recall_10", "map", "ndcg_cut_10"]                    # Qué métricas vamos a usar?
)

**Ejecutar un experimento comparando precisión en varios valores de k**

In [ ]:
rs = pt.Experiment(
    [model_tf, model_bm25, model_dlm],                 # Que estamos evaluando? (modelos)
    dataset.get_topics(),                              # Qué queries usamos?
    dataset.get_qrels(),                               # Qué juicios de relevancia?
    # baseline = 0,
    # perquery = True,
    names=["TF", "BM25", "DLM"],
    eval_metrics=["P"]                                 # Qué métricas vamos a usar?
)

In [ ]:
rs

In [ ]:
df = rs.T
df.columns = list(df.values[:1])
df = df.drop(df[df.index == "name"].index)
df.columns = ["TF", "BM25", "DLM"]
df.rank = [5, 10, 15, 20, 30, 100, 200, 500, 1000]

In [ ]:
df

In [ ]:
sel_rank = [5, 30, 100, 500, 1000]

sel_index = ['P@'+str(x) for x in sel_rank] 
#
tf_performance   = df['TF'].loc[sel_index]
bm25_performance = df['BM25'].loc[sel_index]
dlm_performance  = df['DLM'].loc[sel_index]
#
plt.plot(sel_rank, tf_performance,   'o--', markersize=4, label="TF")
plt.plot(sel_rank, bm25_performance, 'x--', markersize=4, label="BM25")
plt.plot(sel_rank, dlm_performance,  '^--', markersize=4, label="DLM")

#
plt.grid()
plt.xlabel("Rank")
plt.ylabel("P")
#
locs, labels = plt.xticks(sel_rank, rotation=90)
plt.legend(loc=1)
plt.show()

### Tarea 
**1) Ejecutar todo el set de consultas y calcular las principales métricas para k=[1, 5, 10, 50, 100] (cuidado, para R use *recall_10*)**

**2) Ejecute el experimento separando los resultados query por query (perquery = True) y determine si el modelo 'ganador' (puede probar R y P) en promedio es el mismo para todos consideranto k = 10 (tip: calcule en cuántas consultas *gana* cada modelo**